## Data Collection

 **Download** the transcripts from all seasons of *My Little Pony* [here](https://www.kaggle.com/liury123/my-little-pony-transcript).

In [20]:
data_path = '/Users/SheridanVLopez/Desktop/uni/my_little_pony/archive/clean_dialog.csv'
print(data_path) 

/Users/SheridanVLopez/Desktop/uni/my_little_pony/archive/clean_dialog.csv


**Prep**  
   Transform the transcript into a CSV file that has the following columns for each line of dialogue:  
    `episode`, `speaker` , `content`. Content should be clean (only spoken text). 

In [21]:
import pandas as pd
import numpy
from pprint import pprint # pretty print

df = pd.read_csv(data_path)
print(df.head()) 

                         title        writer                           pony  \
0  Friendship is Magic, part 1  Lauren Faust                       Narrator   
1  Friendship is Magic, part 1  Lauren Faust                       Narrator   
2  Friendship is Magic, part 1  Lauren Faust  Narrator and Twilight Sparkle   
3  Friendship is Magic, part 1  Lauren Faust               Twilight Sparkle   
4  Friendship is Magic, part 1  Lauren Faust                   Twinkleshine   

                                              dialog  
0  Once upon a time, in the magical land of Eques...  
1  She vowed that she would shroud the land in et...  
2                                 ...sun and moon...  
3  ...and harmony has been maintained in Equestri...  
4  There you are, Twilight! Moon Dancer is having...  


In [22]:
# remove column 'writer'
df = df.drop(columns=['writer'])
pprint(df.columns.tolist())

['title', 'pony', 'dialog']



Now we have the data we need. But there are a few potential issues that can potentially complicate the problem.
    1. There might be multiple parts for the same episode (ep with same title).
    2. There might be multiple speakers for the same content.
    3. Some dialogs might contain non-spoken texts [sigh] and unicode character instead of UTF-8. E.g. <U+0096>
handle that 

### Problem 1 (Title)


In [23]:
# find unique titles
unique_titles = df['title'].unique()
pprint(unique_titles[:50])

array(['Friendship is Magic, part 1', 'Friendship is Magic, part 2',
       'The Ticket Master', 'Applebuck Season', 'Griffon the Brush Off',
       'Boast Busters', 'Dragonshy', 'Look Before You Sleep',
       'Bridle Gossip', 'Swarm of the Century', 'Winter Wrap Up',
       'Call of the Cutie', 'Fall Weather Friends', 'Suited For Success',
       'Feeling Pinkie Keen', 'Sonic Rainboom', 'Stare Master',
       'The Show Stoppers', 'A Dog and Pony Show',
       "Green Isn't Your Color", 'Over a Barrel', 'A Bird in the Hoof',
       'The Cutie Mark Chronicles', "Owl's Well That Ends Well",
       'Party of One', 'The Best Night Ever',
       'The Return of Harmony Part 1', 'The Return of Harmony Part 2',
       'Lesson Zero', 'Luna Eclipsed', 'Sisterhooves Social',
       'The Cutie Pox', 'May the Best Pet Win!',
       'The Mysterious Mare Do Well', 'Sweet and Elite',
       'Secret of My Excess', "Hearth's Warming Eve",
       'Family Appreciation Day', 'Baby Cakes', 'The Last Roundup


dirty cases, where we have like part X, Part X and other stuff 

The following sequence can be considered: 

1. remove , -
2. convert to lower case
3. remove everything after 'part', including all the space char before

In [24]:
# remove , and - in titles
df['title'] = df['title'].str.replace(',', '').str.replace('-', '')
# convert to lowercase
df['title'] = df['title'].str.lower()
# remove everything after 'part', making sure no space is left before 'part' and at the end
df['title'] = df['title'].str.replace(r'\s*part.*$', '', regex=True)
# make sure no space left at the end
df['title'] = df['title'].str.strip()
# rename column 'title' to 'episode'
df = df.rename(columns={'title': 'episode'})
# print unique titles again
pprint(df['episode'].unique().tolist()[:10])

['friendship is magic',
 'the ticket master',
 'applebuck season',
 'griffon the brush off',
 'boast busters',
 'dragonshy',
 'look before you sleep',
 'bridle gossip',
 'swarm of the century',
 'winter wrap up']


### Problem 2(speaker):


change column `pony` to `speaker`, parse the speakers, and then replace string with list of speakers. or maybe add a copy of dialog for each speaker when multiple speakers. 



In [25]:
# change column pony to speaker
df = df.rename(columns={'pony': 'speaker'})
# convert speaker to lowercase
df['speaker'] = df['speaker'].str.lower()
# parse speaker str into list of speakers, they might be separated by ',', 'and', '&', 'with'
df['speaker'] = df['speaker'].str.replace('&', ',').str.replace(' and ', ',').str.replace(' with ', ',')
df['speaker'] = df['speaker'].str.split(',')
# strip spaces
df['speaker'] = df['speaker'].apply(lambda x: [s.strip() for s in x])
# print speakers and their counts, sort keys by value counts
print(df['speaker'].explode().value_counts().sort_values(ascending=False).to_dict())

{'twilight sparkle': 4769, 'rainbow dash': 3114, 'pinkie pie': 2868, 'applejack': 2779, 'rarity': 2690, 'spike': 2286, 'fluttershy': 2133, 'apple bloom': 1405, 'starlight glimmer': 1180, 'sweetie belle': 980, 'others': 979, 'scootaloo': 956, 'discord': 593, 'trixie': 400, 'princess celestia': 363, 'granny smith': 302, 'big mcintosh': 284, 'princess cadance': 218, 'princess luna': 212, 'maud pie': 212, 'sunburst': 181, 'flim': 178, 'shining armor': 167, 'flam': 164, 'thorax': 158, 'diamond tiara': 155, 'smolder': 146, 'spitfire': 138, 'zecora': 128, 'cheerilee': 119, 'gallus': 113, 'cutie mark crusaders': 106, 'sandbar': 101, 'ocellus': 100, 'cozy glow': 98, 'silverstream': 94, 'yona': 93, 'prince rutherford': 91, 'gilda': 82, 'garble': 81, 'zephyr breeze': 77, 'cranky doodle donkey': 75, 'quibble pants': 74, 'rockhoof': 73, 'young applejack': 72, 'daring do': 67, 'lightning dust': 63, 'queen chrysalis': 62, 'star swirl the bearded': 61, 'mrs. cake': 61, 'ember': 60, 'mayor mare': 60, '

### Problem 3(content):


change column `dialog` to `content`, remove texts embraced by '[ ]', and remove unicode chars.



In [26]:
# change column `dialog to content
df = df.rename(columns={'dialog': 'content'})
# remove texts embraced by [] no extra spaces left
df['content'] = df['content'].str.replace(r'\[.*?\]', '', regex=True)
# remove unicode characters <U+....>
df['content'] = df['content'].str.replace(r'<U\+.*?>', '', regex=True)
# remove extra spaces
df['content'] = df['content'].str.replace(r'\s+', ' ', regex=True)
# strip spaces
df['content'] = df['content'].str.strip()
# make sure contents are strings
df['content'] = df['content'].apply(lambda x: f'{str(x)}')
# print first 10 contents
print(df['content'].tolist()[:10])

["Once upon a time, in the magical land of Equestria, there were two regal sisters who ruled together and created harmony for all the land. To do this, the eldest used her unicorn powers to raise the sun at dawn; the younger brought out the moon to begin the night. Thus, the two sisters maintained balance for their kingdom and their subjects, all the different types of ponies. But as time went on, the younger sister became resentful. The ponies relished and played in the day her elder sister brought forth, but shunned and slept through her beautiful night. One fateful day, the younger unicorn refused to lower the moon to make way for the dawn. The elder sister tried to reason with her, but the bitterness in the young one's heart had transformed her into a wicked mare of darkness: Nightmare Moon.", 'She vowed that she would shroud the land in eternal night. Reluctantly, the elder sister harnessed the most powerful magic known to ponydom: the Elements of Harmony. Using the magic of the E

**Verify your preparation**  
   - Programmatically select 100 random lines from your dataset.  
   - Check: are there any problems with any of these lines?  
   - If so,gotta go back :()

In [27]:

sampled_df = df.sample(n=100, random_state=42)
print(sampled_df)

                                   episode             speaker  \
263                    friendship is magic     [steven magnet]   
11293                spike at your service         [applejack]   
12235            princess twilight sparkle  [twilight sparkle]   
18003    the lost treasure of griffonstone      [rainbow dash]   
4401                    a bird in the hoof  [twilight sparkle]   
...                                    ...                 ...   
4528             the cutie mark chronicles         [scootaloo]   
24198                  dungeons & discords    [spike, discord]   
7615   the super speedy cider squeezy 6000        [flim, flam]   
22856                applejack's "day" off         [applejack]   
28490                  fame and misfortune         [applejack]   

                                                 content  
263                          What a world, what a world.  
11293  Thanks for walkin' me home, Spike. That was mi...  
12235  That would be nice, Spi

### Task 3. annotation
add an annotation for who the speaker is addressing.code will create an annotated version of the csv file which contains an additional column “addressee” indicating who is being addressed.

approach: the character who’s dialogue immediately follows the speaker is the addressee.


In [28]:

# spearker(s) is a list, so the addressee should be a list too
# we can use a function to solve this

def find_addressee(row, df):
    if row.name == len(df) - 1:  # if it's the last row, no addressee
        return 'None' 
    next_row = df.iloc[row.name + 1]
    if row['episode'] != next_row['episode']:  # if episode changes, no addressee 
        return 'None'
    return next_row['speaker'] #ly shifting the speaker column up by 1 within the same episode

df['addressee'] = df.apply(lambda row: find_addressee(row, df), axis=1)
# print the dataframe to check
print(df.head(10))
# save to csv
df.to_csv('/Users/SheridanVLopez/Desktop/uni/my_little_pony/archive/clean_dialog_with_addressee.csv', index=False)
print("Saved to /Users/SheridanVLopez/Desktop/uni/my_little_pony/archive/clean_dialog_with_addressee.csv")
# print the dataframe to check
print(df.head(10))

               episode                       speaker  \
0  friendship is magic                    [narrator]   
1  friendship is magic                    [narrator]   
2  friendship is magic  [narrator, twilight sparkle]   
3  friendship is magic            [twilight sparkle]   
4  friendship is magic                [twinkleshine]   
5  friendship is magic            [twilight sparkle]   
6  friendship is magic                [twinkleshine]   
7  friendship is magic            [twilight sparkle]   
8  friendship is magic                       [spike]   
9  friendship is magic            [twilight sparkle]   

                                             content  \
0  Once upon a time, in the magical land of Eques...   
1  She vowed that she would shroud the land in et...   
2                                 ...sun and moon...   
3  ...and harmony has been maintained in Equestri...   
4  There you are, Twilight! Moon Dancer is having...   
5  Oh, sorry, girls... I've got a lot of studyi